In [1]:
#from preprocessor import Dataset, pad_vec_sequences, pad_sent_sequences
from sklearn import preprocessing
from sklearn import cross_validation

import os
os.environ['KERAS_BACKEND']='tensorflow' # set backend

In [2]:
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, merge, GRU
from keras.utils import np_utils, generic_utils

Using TensorFlow backend.


In [5]:
# look into preprocessor.py file for details
from preprocessor import Tensor_Sequence, accuracy, indicator_to_matrix

In [6]:
yelp_data = Tensor_Sequence("yelp_academic_dataset_review.json", "text", "stars", 100)

In [7]:
# data split
x_train, x_test, y_train, y_test = cross_validation.train_test_split(yelp_data.X_doc_seq,yelp_data.Y_doc_seq,test_size=0.2)

# create appropirate matrix (hot encoded) response
y_train, y_test = [indicator_to_matrix(x,yelp_data.docs_label_index)  for x in (y_train, y_test)]

In [16]:
maxlen = yelp_data.maxlen
hidden_dim = 128
nb_classes = len(yelp_data.docs_label_index)

In [13]:
#In train.py
# separate model specification from compilation for clarity and error tracing
sequence = Input(shape=(maxlen,len(yelp_data.docs_vocab)), dtype='float32')
forwards = LSTM(hidden_dim,dropout_W=0.1,dropout_U=0.1)(sequence)
backwards = LSTM(hidden_dim,dropout_W=0.1,dropout_U=0.1,go_backwards=True)(sequence)
merged = merge([forwards, backwards], mode='concat', concat_axis=-1)
after_dp = Dropout(0.1)(merged)
output = Dense(nb_classes, activation='softmax')(after_dp)
model = Model(input=sequence, output=output)

In [14]:
model.compile('adam', 'categorical_crossentropy')

In [15]:
# model specs
batch_size = 32
num_epoch = 10

# train model
model.fit(x_train, y_train,
          batch_size=batch_size,
          nb_epoch=num_epoch,
          validation_data=[x_test, y_test])

Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - 5s - loss: 1.6013 - val_loss: 1.5809
Epoch 2/10
80/80 [==============================] - 3s - loss: 1.5465 - val_loss: 1.5423
Epoch 3/10
80/80 [==============================] - 3s - loss: 1.4800 - val_loss: 1.4845
Epoch 4/10
80/80 [==============================] - 3s - loss: 1.4008 - val_loss: 1.4025
Epoch 5/10
80/80 [==============================] - 3s - loss: 1.2830 - val_loss: 1.3062
Epoch 6/10
80/80 [==============================] - 3s - loss: 1.1693 - val_loss: 1.2460
Epoch 7/10
80/80 [==============================] - 3s - loss: 1.1053 - val_loss: 1.2223
Epoch 8/10
80/80 [==============================] - 3s - loss: 1.0018 - val_loss: 1.2031
Epoch 9/10
80/80 [==============================] - 3s - loss: 0.8315 - val_loss: 1.2039
Epoch 10/10
80/80 [==============================] - 3s - loss: 0.6670 - val_loss: 1.1814


In [69]:
model.save('yelp_lstm_hidden_epocs.h5')

In [ ]:
from keras.models import load_model
current_model = load_model('yelp_lstm_hidden_epocs.h5')